In [105]:
#notebook for testing the search of song in a wav file

## Load everything
%matplotlib inline
import pdb
import sys
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
matplotlib.style.use('ggplot')

# Set up Computer and paths
comp_name=socket.gethostname()
print 'Computer: ' + comp_name

if 'lintu' in comp_name:
    sys.path.append('/mnt/cube/earneodo/repos/soundflow/sound_tools')
    experiment_folder = os.path.join('/mnt/cube/leisenman/')
    raw_data_folder = os.path.join(experiment_folder, 'song_data')

    
#grab soundtools
from soundtools import WavData, Chunk
    
#file structure
bird_id = 'b1237'
sess = 20160630
run = 1

# Setup 
raw_file_folder = os.path.join(raw_data_folder, bird_id, str(sess).zfill(3))
raw_file_name = str(run).zfill(3) + '.wav'
raw_file_path = os.path.join(raw_file_folder, raw_file_name)

bout_file_folder = os.path.join(raw_file_folder, 'bouts')

if not os.path.isdir(bout_file_folder):
    os.makedirs(bout_file_folder)
    

# preparations and parameters
#the whole sound
all_sound = WavData(raw_file_path)
s_f = all_sound.s_f

#The band where we expect most of the energy of zf song to be in
filt_lo = 10000 #Hz
filt_hi = 500 #Hz
hp_b, hp_a = sg.butter(4, filt_hi/(s_f/2.), btype='high')
lp_b, lp_a = sg.butter(4, filt_lo/(s_f/2.), btype='low')


def band_pass_filter(chunk, hp_b, hp_a, lp_b, lp_a):    
    chunk_hi = sg.filtfilt(hp_b, hp_a, chunk)
    chunk_filt = sg.filtfilt(lp_b, lp_a, chunk_hi)
    return chunk_filt

def seconds(t):
    if type(t)==str:
        return np.dot(np.fromstring(t, sep=':'), np.array([3600.,60.,1.]))
    else:
        return t

Computer: lintu.ucsd.edu


In [108]:
#list of start, end points (in 'h:m:s'):
segments = [['1:12:04', '1:12:31'],
            ['1:18:35', '1:19:00'],
            ['1:19:47', '1:20:10'],
            ['1:21:12', '1:21:41'],
            ['1:22:28', '1:22:59'],
            ['1:24:33', '1:25:03'],
            ['1:28:09', '1:28:33'],
            ['1:29:38', '1:30:04'],
            ['1:31:24', '1:31:49'],
            ['1:39:49', '1:40:13']
            ]


for segment in segments:
    #print start, end
    start_seconds = int(seconds(segment[0]))
    start = int(seconds(segment[0]) * all_sound.s_f)
    end = int(seconds(segment[1]) * all_sound.s_f)
    
    #Make a chunk with each segment and record it to a file
    bout_file_name = str(run).zfill(3) + '_' + str(int(start_seconds)).zfill(5) + '.wav'
    bout_file_path = os.path.join(bout_file_folder, bout_file_name)
    bout_chunk = Chunk(all_sound, segment = [start, end])
    # save the chunk to a data file
    #filter for debugging
    #bout_chunk.apply_filter(band_pass_filter, hp_b, hp_a, lp_b, lp_a)
    data_out = np.array(bout_chunk.data, dtype=np.int16)[0]
    packed_data = struct.pack('<' + str((end - start)) + 'h', *data_out)
    print bout_file_path
    bout_out = wave.open(bout_file_path, 'w')
    bout_out.setparams((all_sound.n_chans, 2, all_sound.s_f, 0, 'NONE', 'not compressed'))
    bout_out.writeframes(data_out)
    bout_out.close()    

/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_04324.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_04715.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_04787.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_04872.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_04948.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_05073.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_05289.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_05378.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_05484.wav
/mnt/cube/leisenman/song_data/b1237/20160630/bouts/001_05989.wav


In [6]:
all_sound.n_samples/all_sound.s_f/3600

11

In [27]:
start/all_sound.s_f/3600

19

In [9]:
all_sound.raw.getparams()

(1, 2, 44100, 1828812800, 'NONE', 'not compressed')

In [112]:
segments = [['1:12:04', '1:12:31'],
            ['1:18:35', '1:19:00'],
            ['1:19:47', '1:20:10'],
            ['1:21:12', '1:21:41'],
            ['1:22:28', '1:22:59'],
            ['1:24:33', '1:25:03'],
            ['1:28:09', '1:28:33'],
            ['1:29:38', '1:30:04'],
            ['1:31:24', '1:31:49'],
            ['1:39:49', '1:40:13']
            ]


for segment in segments:
    #print start, end
    start_seconds = int(seconds(segment[0]))
    start = int(seconds(segment[0]) * all_sound.s_f)
    end = int(seconds(segment[1]) * all_sound.s_f)
    print (end-start)/all_sound.s_f

27
25
23
29
31
30
24
26
25
24


In [111]:
s_f

44100